<font size="6"><code style="background:teal;color:white">A/B тестрирование</code></font>

<div class="alert alert-block alert-warning"><b>Задание:</b> В абстрактном приложении с подпиской для чтения историй (по типу Hooked или Yarn) был проведен A/B-тест. Двум группам игроков были предложены разные стартовые истории при первом запуске приложения. Вам выпала честь оценить результаты проведенного A/B-теста. Какую группу вы порекомендуете оставить и почему? Данные о сессиях пользователей и совершенных ими покупках (подписках) представлены в файлах sessions.csv и payments.csv. .</div>

Для проверки результатов успешности тестирования будем исследовать следующие метрики в контрольной и тестируемой группах:  
- Convertion
- Средний чек 
- ARPU 
- ARPPU

Хотелось бы еще посчитать удержание и сделать когортный анализ, но у нас недостаточно данных (отсуствует день регистрации)

**<mark>ЭТАП 1</mark>  Загружаем необходимые для работы библиотеки, сами данные, изучаем и подготавливаем их для последующего анализа.**

In [1]:
import pandas as pd
from scipy import stats
from statsmodels.stats.proportion import proportion_confint
from statsmodels.stats.weightstats import ztest
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly import graph_objects as go
init_notebook_mode(connected=True)
from plotly.subplots import make_subplots

In [2]:
# Загрузим данные с сессиями
sessions = pd.read_csv('sessions.csv')#если csv файлы лежат в другой папке нежеле данный ipynb файл необходим полный путь
sessions.columns = ['timestamp', 'user_id', 'group', 'event'] #переименуем для удобства названия колонок
sessions

,timestamp,user_id,group,event
0,1514768820,ab3a0b4e-6286-4748-90a1-e7d58273594d,05_12FTD_control_group,session start
1,1514782260,e64698c1-0168-4523-9592-f5226e63b9e6,05_12FTD_control_group,session start
2,1514811780,a8436498-ca01-4f09-9bac-de1bf217e00e,05_12FTD_control_group,session start
3,1514831460,6e5830b2-85b7-44b3-b7dd-3cb2e18c2897,05_12FTD_test_group,session start
4,1514831940,3b1809fe-e495-49e2-a8d1-b888d0a6948e,05_12FTD_control_group,session start
...,...,...,...,...
8071,1517431980,8a64d64b-db9a-4993-980c-6b3c8a51d9a1,05_12FTD_control_group,session start
8072,1517431980,d19e10b4-e929-45c9-80b6-09b288b62cfb,05_12FTD_test_group,session start
8073,1517432160,97c42130-df1f-4127-af7e-6b0e3e14a207,05_12FTD_test_group,session start
8074,1517432160,d21e2ee9-290d-4d1f-b749-3558db98d32b,05_12FTD_test_group,session start


In [3]:
sessions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8076 entries, 0 to 8075
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   timestamp  8076 non-null   int64 
 1   user_id    8076 non-null   object
 2   group      8076 non-null   object
 3   event      8076 non-null   object
dtypes: int64(1), object(3)
memory usage: 252.5+ KB


In [4]:
sessions.drop_duplicates().info() #проверим наличие дубликатов в данных и посмотрим информацию о нем

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8076 entries, 0 to 8075
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   timestamp  8076 non-null   int64 
 1   user_id    8076 non-null   object
 2   group      8076 non-null   object
 3   event      8076 non-null   object
dtypes: int64(1), object(3)
memory usage: 315.5+ KB


In [5]:
# Преобразуем дату к привычному нам формату:
sessions['date'] = sessions.apply(lambda x: pd.Timestamp(x['timestamp'], unit='s'), axis = 1)
sessions

,timestamp,user_id,group,event,date
0,1514768820,ab3a0b4e-6286-4748-90a1-e7d58273594d,05_12FTD_control_group,session start,2018-01-01 01:07:00
1,1514782260,e64698c1-0168-4523-9592-f5226e63b9e6,05_12FTD_control_group,session start,2018-01-01 04:51:00
2,1514811780,a8436498-ca01-4f09-9bac-de1bf217e00e,05_12FTD_control_group,session start,2018-01-01 13:03:00
3,1514831460,6e5830b2-85b7-44b3-b7dd-3cb2e18c2897,05_12FTD_test_group,session start,2018-01-01 18:31:00
4,1514831940,3b1809fe-e495-49e2-a8d1-b888d0a6948e,05_12FTD_control_group,session start,2018-01-01 18:39:00
...,...,...,...,...,...
8071,1517431980,8a64d64b-db9a-4993-980c-6b3c8a51d9a1,05_12FTD_control_group,session start,2018-01-31 20:53:00
8072,1517431980,d19e10b4-e929-45c9-80b6-09b288b62cfb,05_12FTD_test_group,session start,2018-01-31 20:53:00
8073,1517432160,97c42130-df1f-4127-af7e-6b0e3e14a207,05_12FTD_test_group,session start,2018-01-31 20:56:00
8074,1517432160,d21e2ee9-290d-4d1f-b749-3558db98d32b,05_12FTD_test_group,session start,2018-01-31 20:56:00


Мы видим, что тестирование проводилось с 1 января 2018 по 31 января 2018 года, то есть в течение месяца.

In [6]:
# Посмотрим какие события присутствуют в датафрейме с сессиями
sessions.event.unique()

array(['session start'], dtype=object)

In [7]:
# Посмотрим какие группы присутствуют в датафрейме с сессиями
sessions.group.unique()

array(['05_12FTD_control_group', '05_12FTD_test_group'], dtype=object)

In [8]:
# Загрузим данные с платежами
payments = pd.read_csv('payments.csv')
payments.columns = ['timestamp', 'user_id', 'event', 'is_trial', 'price'] #переименуем для удобства названия колонок
payments

,timestamp,user_id,event,is_trial,price
0,1514754000,59c856ca5638e80001b8bad2,payment,False,2.99
1,1514755080,32eb44a1-4c44-49e1-8322-253ef00ac52c,payment,True,0.00
2,1514756460,acfaee6d-abe2-4c8c-a243-98331eb5a736,payment,True,0.00
3,1514762280,58c2e0bf-6fd6-4f79-bcc1-9c21e2ef0e0a,payment,True,0.00
4,1514763480,1cd8ccf6-09a0-4dc8-ab9b-786c392ca3e4,payment,True,0.00
...,...,...,...,...,...
4260,1517429700,59b60e2b5638e800019776a7,payment,False,2.99
4261,1517430900,b9e6387f-9a66-4951-b569-6f42185d52c0,payment,True,0.00
4262,1517431080,80378737-946c-4309-b2ff-bdef513ff815,payment,True,0.00
4263,1517431200,402e5074-360b-416d-b3fc-d7c29c9ab2c3,payment,False,7.99


In [9]:
payments.drop_duplicates().info()  #проверим наличие дубликатов в данных и посмотрим информацию о нем

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4265 entries, 0 to 4264
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   timestamp  4265 non-null   int64  
 1   user_id    4265 non-null   object 
 2   event      4265 non-null   object 
 3   is_trial   4265 non-null   bool   
 4   price      4265 non-null   float64
dtypes: bool(1), float64(1), int64(1), object(2)
memory usage: 170.8+ KB


In [10]:
# Преобразуем дату к привычному нам формату:
payments['date'] = payments.apply(lambda x: pd.Timestamp(x['timestamp'], unit='s'), axis = 1)
payments

,timestamp,user_id,event,is_trial,price,date
0,1514754000,59c856ca5638e80001b8bad2,payment,False,2.99,2017-12-31 21:00:00
1,1514755080,32eb44a1-4c44-49e1-8322-253ef00ac52c,payment,True,0.00,2017-12-31 21:18:00
2,1514756460,acfaee6d-abe2-4c8c-a243-98331eb5a736,payment,True,0.00,2017-12-31 21:41:00
3,1514762280,58c2e0bf-6fd6-4f79-bcc1-9c21e2ef0e0a,payment,True,0.00,2017-12-31 23:18:00
4,1514763480,1cd8ccf6-09a0-4dc8-ab9b-786c392ca3e4,payment,True,0.00,2017-12-31 23:38:00
...,...,...,...,...,...,...
4260,1517429700,59b60e2b5638e800019776a7,payment,False,2.99,2018-01-31 20:15:00
4261,1517430900,b9e6387f-9a66-4951-b569-6f42185d52c0,payment,True,0.00,2018-01-31 20:35:00
4262,1517431080,80378737-946c-4309-b2ff-bdef513ff815,payment,True,0.00,2018-01-31 20:38:00
4263,1517431200,402e5074-360b-416d-b3fc-d7c29c9ab2c3,payment,False,7.99,2018-01-31 20:40:00


In [11]:
# Посмотрим какие цены существуют в данных с платежами:
payments.price.unique()

array([ 2.99,  0.  ,  7.99, 39.99, 15.98])

Мы видим, что дата в платежах начинается раньше, чем начало тестирования. Поэтому оставим только те записи, где время платежей попадает в интервал проведения тсетирования

In [12]:
new_payments = payments[(payments['timestamp'] >= sessions['timestamp'].min())&\
                        (payments['timestamp'] <= sessions['timestamp'].max())]
new_payments

,timestamp,user_id,event,is_trial,price,date
7,1514769000,34cd4ba1-0a06-41ef-a9c6-d3d1366f16d4,payment,False,7.99,2018-01-01 01:10:00
8,1514773980,6b0619b7-e212-4e57-8787-2606b49671be,payment,True,0.00,2018-01-01 02:33:00
9,1514774400,59da09415638e80001146c1f,payment,False,2.99,2018-01-01 02:40:00
10,1514775720,e5ea91e7-3c86-4b60-b942-3b7a00e1dfd6,payment,True,0.00,2018-01-01 03:02:00
11,1514779020,e95d9872-e67b-4882-a962-e44df722b033,payment,True,0.00,2018-01-01 03:57:00
...,...,...,...,...,...,...
4260,1517429700,59b60e2b5638e800019776a7,payment,False,2.99,2018-01-31 20:15:00
4261,1517430900,b9e6387f-9a66-4951-b569-6f42185d52c0,payment,True,0.00,2018-01-31 20:35:00
4262,1517431080,80378737-946c-4309-b2ff-bdef513ff815,payment,True,0.00,2018-01-31 20:38:00
4263,1517431200,402e5074-360b-416d-b3fc-d7c29c9ab2c3,payment,False,7.99,2018-01-31 20:40:00


Видим, что не попали первые 6 пользователей в наш новый датафрейм по платежам. Ради интереса посмотрим присутствуют ли ID этих пользователей в данных с сессиями:

In [13]:
sessions[sessions['user_id'].isin(payments.iloc[0:7].user_id)]

,timestamp,user_id,group,event,date
1199,1516397580,de7cd31c-36b4-42d8-bcc9-74642447b58c,05_12FTD_test_group,session start,2018-01-19 21:33:00
1339,1516407960,de7cd31c-36b4-42d8-bcc9-74642447b58c,05_12FTD_test_group,session start,2018-01-20 00:26:00
1365,1516410300,de7cd31c-36b4-42d8-bcc9-74642447b58c,05_12FTD_test_group,session start,2018-01-20 01:05:00
1451,1516418280,de7cd31c-36b4-42d8-bcc9-74642447b58c,05_12FTD_test_group,session start,2018-01-20 03:18:00
1473,1516419480,de7cd31c-36b4-42d8-bcc9-74642447b58c,05_12FTD_test_group,session start,2018-01-20 03:38:00


<span style="color:teal"> <b>В тестирование попали пользователи, которые уже были учтены в системе ранее. Как правило, для тестов стараются отбирать новых пользователей во избежание "сюрпризов" и негативного опыта использования приложения. Однако, в данном случае, тестируется стартовая история при первом запуске приложения, которая (предположение) меняется с определенной частотой (возможно каждый день). Поэтому данный момент не является критичным.</b></span>

In [14]:
# Посмотрим на количество уникальных пользователей в обоих датафреймах:
print('Количество уникальных пользователей в данных с сессиями: {}'.format(sessions.user_id.nunique()))
print('Количество уникальных пользователей в данных с платежами: {}'.format(new_payments.user_id.nunique()))

Количество уникальных пользователей в данных с сессиями: 2218
Количество уникальных пользователей в данных с платежами: 2541


В платежи попали пользователи не участвующие в проводимом тестировании, делаем преобразования:

In [15]:
#Оставим в платежах только те записи, где пользователи участвуют в тестировании согласно датафрейму о сессиях
new_payments = new_payments[new_payments['user_id'].isin(sessions.user_id.unique())]

In [16]:
# Проверим количество уникальных пользователей, участвующих в данном тестировании, в данных с платежами:
new_payments.user_id.nunique()

130

В дальнейшем нам понадобится строить рассчеты на основании ID пользователя согласно принадлежности к группе. Контрольная группа будет иметь идентификатор А в названии, тестовая - В.

In [17]:
A_usersID_sessions = sessions[sessions.group == '05_12FTD_control_group'].user_id.unique()
B_usersID_sessions = sessions[sessions.group == '05_12FTD_test_group'].user_id.unique()

In [18]:
# Добавим идентификатор группы в данные с платежами
new_payments = new_payments.copy()
new_payments['group'] = new_payments.apply(lambda x: 'A' if x['user_id'] in A_usersID_sessions else 'B', axis=1)
new_payments

,timestamp,user_id,event,is_trial,price,date,group
38,1514821680,68230d15-2408-4c0b-b5aa-62a4d82f3486,payment,True,0.00,2018-01-01 15:48:00,A
42,1514828100,c9a6e320-3d41-4547-961b-ba58ce7797d8,payment,True,0.00,2018-01-01 17:35:00,B
65,1514848860,241537da-08da-4ca8-a2ba-8bb737ee9902,payment,True,0.00,2018-01-01 23:21:00,B
68,1514852640,27619640-07fd-4f12-81ff-b9f973cb7179,payment,False,39.99,2018-01-02 00:24:00,B
69,1514852640,27619640-07fd-4f12-81ff-b9f973cb7179,payment,True,0.00,2018-01-02 00:24:00,B
...,...,...,...,...,...,...,...
4198,1517343420,5fc42716-2bfe-4a8e-8476-e198bee2ed3e,payment,True,0.00,2018-01-30 20:17:00,B
4207,1517352300,fbb2f6b2-2f36-41a3-9d10-98142ae12fad,payment,False,7.99,2018-01-30 22:45:00,A
4221,1517370120,9bb2a648-f9be-445e-b2ff-59826046051e,payment,True,0.00,2018-01-31 03:42:00,B
4228,1517385900,d98ea24c-85cb-4ea5-ab9d-22dbd1fea1b8,payment,False,7.99,2018-01-31 08:05:00,A


**<mark>ЭТАП 2</mark>  Рассчитываем показатели.** 

<span style="color:teal"> <b>Рассчитаем конверсию по группам сначала в trial, а потом в покупку услуги.</b></span>

In [19]:
ab_total=pd.DataFrame([len(A_usersID_sessions), len(B_usersID_sessions)], index = ['A','B'], columns = ['users'])
ab_total

,users
A,1055
B,1163


In [20]:
ab_total.insert(1,"trial", [new_payments[(new_payments.group == 'A')&(new_payments.is_trial == True)].user_id.nunique(), \
                         new_payments[(new_payments.group == 'B')&(new_payments.is_trial == True)].user_id.nunique()], True)
ab_total['trial_users_conv'] = ab_total.trial/ab_total.users
ab_total.insert(2,"buyers", [new_payments[(new_payments.group == 'A')&(new_payments.is_trial == False)].user_id.nunique(), \
                         new_payments[(new_payments.group == 'B')&(new_payments.is_trial == False)].user_id.nunique()], True)
ab_total['buyers_trial_conv'] = ab_total.buyers/ab_total.trial
ab_total['buyers_users_conv'] = ab_total.buyers/ab_total.users
ab_total

,users,trial,buyers,trial_users_conv,buyers_trial_conv,buyers_users_conv
A,1055,56,49,0.053081,0.875000,0.046445
B,1163,57,45,0.049011,0.789474,0.038693


Видим, что конверсия в контрольной группе на всех этапах выше.

<span style="color:teal"> <b>Рассчитаем показатели монетизации.</b></span>

In [21]:
ab_total.insert(6, "amount", [new_payments[new_payments.group == 'A'].price.sum(), \
                              new_payments[new_payments.group == 'B'].price.sum()], True) # Выручка в группе
ab_total.insert(7, "avg_bill", [new_payments[new_payments.group == 'A'].price.sum()/new_payments[(new_payments.group == 'A')&\
                                (new_payments.is_trial == False)].price.count(), \
                               new_payments[new_payments.group == 'B'].price.sum()/new_payments[(new_payments.group == 'B')&\
                                (new_payments.is_trial == False)].price.count()], True) # Средний чек в группе
ab_total['ARPU'] = ab_total.amount/ab_total.users
ab_total['ARPPU'] = ab_total.amount/ab_total.buyers
ab_total

,users,trial,buyers,trial_users_conv,buyers_trial_conv,buyers_users_conv,amount,avg_bill,ARPU,ARPPU
A,1055,56,49,0.053081,0.875000,0.046445,519.35,7.99,0.492275,10.598980
B,1163,57,45,0.049011,0.789474,0.038693,575.40,9.59,0.494755,12.786667


<div class="alert alert-block alert-success">
<b>Вывод:</b> Согласно нашим предварительным выводам показатели конверсии в тестовой группе на 17% меньше, чем в контрольной. При этом средний чек увеличился на 20%, ARPU практически остался неизменным, ARPPU увеличился на 21%. Полученные результаты свидетельствуют о том, что скорее всего мы стали предоставлять стартовые истории именно нашей целевой аудитории, за которые они готовы платить.
</div>

**<mark>ЭТАП 3</mark>  Проверяем статистическую значимость полученных результатов.**

Конверсия — бинарная величина (пользователь либо купил, либо нет), поэтому для подсчёта значимости нужно использовать Z-test.  
Мы будем использовать statsmodels.stats.weightstats.ztest, куда нужно передать две последовательности пользовательских конверсий (то есть массивы нулей и единиц). Для этого преобразуем наши датафреймы.

In [22]:
a_data = pd.DataFrame(data = A_usersID_sessions, columns = ['user_id']) # Датафрейм с user_id группы А
a_data = a_data.merge(new_payments[(new_payments.group == 'A')&(new_payments.is_trial == False)].\
                      groupby('user_id')['price'].agg('sum'), how='left', on='user_id') # Присоединили сумму покупки по ID
a_data['is_buyer'] = a_data.apply(lambda x: 1 if x['price']>0 else 0, axis = 1) # Добавили новый признак
a_data.head()

,user_id,price,is_buyer
0,ab3a0b4e-6286-4748-90a1-e7d58273594d,NaN,0
1,e64698c1-0168-4523-9592-f5226e63b9e6,NaN,0
2,a8436498-ca01-4f09-9bac-de1bf217e00e,NaN,0
3,3b1809fe-e495-49e2-a8d1-b888d0a6948e,NaN,0
4,36782654-61d9-4bf9-915d-ba1b268e18de,NaN,0


In [23]:
b_data = pd.DataFrame(data = B_usersID_sessions, columns = ['user_id']) # Датафрейм с user_id группы В
b_data = b_data.merge(new_payments[(new_payments.group == 'B')&(new_payments.is_trial == False)].\
                      groupby('user_id')['price'].agg('sum'), how='left', on='user_id') # Присоединили сумму покупки по ID
b_data['is_buyer'] = b_data.apply(lambda x: 1 if x['price']>0 else 0, axis = 1) # Добавили новый признак
b_data.head()

,user_id,price,is_buyer
0,6e5830b2-85b7-44b3-b7dd-3cb2e18c2897,NaN,0
1,f9dc3449-b4f7-48ed-9642-e4b76e70a21f,NaN,0
2,da339891-3a3d-4c84-a3f8-ed552f8ef260,NaN,0
3,77878fb7-c734-4fbc-9f62-e96bfc7d1301,NaN,0
4,13db7147-0d51-4a53-ae78-07c947914449,NaN,0


In [24]:
ztest(a_data.is_buyer, b_data.is_buyer)

(0.9048564072215693, 0.36554146392566655)

<span style="color:teal"> <b>Z-test выдаёт два параметра: z-статистику и p-value. Нам нужно как раз p-value, которое в данном случае превышает пороговое значение 0.05, что не дает нам оснований отклонить нулевую гипотезу и признать полученные различия в конверсии статистически значимыми</b></span>

**Считаем изменения среднего чека, ARPU и ARPPU**  

Рассмотрим теперь различия среднего чека и ARPU. Очевидно, что и средний чек, и ARPU распределены ненормально, так как у нас ограниченное количество цен на подписки (предполагаем что платим за подписки на определенный срок). Давайте визуализируем это и проверим с помощью статистического теста Шапиро-Вилка для распределения средних чеков и покупок:

In [25]:
trace1 = go.Histogram(
            x = a_data.price.dropna(),
            name='Контрольная группа А', 
            hovertemplate="%{x}, <br>%{y} <extra></extra>",
            marker_color='teal',
            opacity=0.75
            )
trace2 = go.Histogram(
            x=b_data.price.dropna(),
            name='Тестируемая группа В',
            hovertemplate="%{x}, <br>%{y} <extra></extra>",
            marker_color='salmon',
            opacity=0.75
            )

data = [trace1, trace2]

layout=go.Layout(
    title_text='Гистограмма распределения чеков на платящих пользователей ', 
    xaxis_title_text='Чек, $', # Делаем предположение, что суммы у нас в USD,тточно не указано
    yaxis_title_text='Количество пользователей', 
    bargap=0.2, 
    bargroupgap=0.1 
)

fig=go.Figure(data = data, layout = layout)

fig.show()
print('Shapiro-Wilk p-value для группы А: ', stats.shapiro(a_data.price.dropna())[1])
print('Shapiro-Wilk p-value для группы B: ', stats.shapiro(b_data.price.dropna())[1])

Shapiro-Wilk p-value для группы А:  4.698272326741915e-10
Shapiro-Wilk p-value для группы B:  2.886953176517437e-10


In [26]:
trace1 = go.Histogram(
            x = a_data.price.fillna(0),
            name='Контрольная группа А', 
            hovertemplate="%{x}, <br>%{y} <extra></extra>",
            marker_color='teal',
            opacity=0.75
            )
trace2 = go.Histogram(
            x=b_data.price.fillna(0),
            name='Тестируемая группа В',
            hovertemplate="%{x}, <br>%{y} <extra></extra>",
            marker_color='salmon',
            opacity=0.75
            )

data = [trace1, trace2]

layout=go.Layout(
    title_text='Гистограмма распределения выручки на пользователей ', 
    xaxis_title_text='Выручка, $', # Делаем предположение, что суммы у нас в USD,тточно не указано
    yaxis_title_text='Количество пользователей', 
    bargap=0.2, 
    bargroupgap=0.1 
)

fig=go.Figure(data = data, layout = layout)

fig.show()
print('Shapiro-Wilk p-value для группы А: ', stats.shapiro(a_data.price.fillna(0))[1])
print('Shapiro-Wilk p-value для группы B: ', stats.shapiro(b_data.price.fillna(0))[1])

Shapiro-Wilk p-value для группы А:  0.0
Shapiro-Wilk p-value для группы B:  0.0


In [27]:
a_payments = new_payments[(new_payments.group == 'A')&(new_payments.is_trial == False)].\
                groupby('user_id')['price'].agg('sum').reset_index()
b_payments = new_payments[(new_payments.group == 'B')&(new_payments.is_trial == False)].\
                groupby('user_id')['price'].agg('sum').reset_index()
trace1 = go.Histogram(
            x = a_payments.price,
            name='Контрольная группа А', 
            hovertemplate="%{x}, <br>%{y} <extra></extra>",
            marker_color='teal',
            opacity=0.75
            )
trace2 = go.Histogram(
            x=b_payments.price,
            name='Тестируемая группа В',
            hovertemplate="%{x}, <br>%{y} <extra></extra>",
            marker_color='salmon',
            opacity=0.75
            )

data = [trace1, trace2]

layout=go.Layout(
    title_text='Гистограмма распределения выручки на платящего пользователей ', 
    xaxis_title_text='Выручка, $', # Делаем предположение, что суммы у нас в USD,тточно не указано
    yaxis_title_text='Количество пользователей', 
    bargap=0.2, 
    bargroupgap=0.1 
)

fig=go.Figure(data = data, layout = layout)

fig.show()
print('Shapiro-Wilk p-value для группы А: ', stats.shapiro(a_payments.price)[1])
print('Shapiro-Wilk p-value для группы B: ', stats.shapiro(b_payments.price)[1])

Shapiro-Wilk p-value для группы А:  4.698272326741915e-10
Shapiro-Wilk p-value для группы B:  2.886953176517437e-10


<span style="color:teal"> <b>И визуальный анализ, и тест Шапиро-Вилка на нормальность говорят нам, что распределение не является нормальным (stats.shapiro выдаёт два числа, второе — p-value сравнения нормального распределения и нашего).</b></span>

Так как распределения признаков не бинарные (0-1) и не нормальные, то нужно использовать для сравнения средних не только T-test, но и непараметрический U-тест Манна-Уитни.  

U-test скажет нам, различаются ли сами распределения, а T-test покажет, различаются ли средние этих распределений. При сравнении средних величин (например, ARPU) обычно важнее показатель p-value для Т-теста. Если он меньше 0.05, а p-value теста Манна-Уитни больше 0.05, то это все равно говорит о значимых различиях показателей.

**Сравним средние чеки:**

In [28]:
print(stats.mannwhitneyu(a_data.price.dropna(), b_data.price.dropna()),
stats.ttest_ind(a_data.price.dropna(), b_data.price.dropna())
      , sep = '\n')

MannwhitneyuResult(statistic=1024.5, pvalue=0.23808981068428547)
Ttest_indResult(statistic=-1.5236828375885085, pvalue=0.1310181363993306)


Как видим, мы не получили статистически значимых различий средних чеков для групп ни по одному из параметров

**Сравним ARPU:**

In [29]:
print(stats.mannwhitneyu(a_data.price.fillna(0), b_data.price.fillna(0)),
stats.ttest_ind(a_data.price.fillna(0), b_data.price.fillna(0))
      , sep = '\n')

MannwhitneyuResult(statistic=608804.5, pvalue=0.18679491104337975)
Ttest_indResult(statistic=-0.021203576903127387, pvalue=0.983085169909874)


Как видим, мы не получили статистически значимых различий ARPU для групп ни по одному из параметров

**Сравним ARPPU:**

In [30]:
print(stats.mannwhitneyu(a_payments.price, a_payments.price),
stats.ttest_ind(a_payments.price, a_payments.price)
      , sep = '\n')

MannwhitneyuResult(statistic=1200.5, pvalue=0.49823206535942105)
Ttest_indResult(statistic=0.0, pvalue=1.0)


Как видим, мы не получили статистически значимых различий ARPPU для групп ни по одному из параметров

<div class="alert alert-block alert-success">
<b>Вывод:</b> Ни по одной из рассчитываемых метрик нам не удалось получить статистически значимые различия, следовательно, результаты тестирования подвержены сомнению.
</div>

**<mark>ЭТАП 4</mark>  Визуализируем полученные результаты.**

In [31]:
fig = make_subplots(rows = 2, cols = 4, 
                   subplot_titles = ("users", "buyers", "conversion", "revenue", "avg_bill", "ARPU", "ARPPU"))

fig.add_trace(go.Bar(x = ab_total.index, 
                     y = ab_total.users,
                     name = "users",
                     text = ab_total.users,
                     hovertemplate="%{x}, <br>%{y} <extra></extra>",
                     textposition='auto',
                     marker=dict(color=['teal', 'salmon'])
                    ), row = 1, col = 1)
fig.add_trace(go.Bar(x = ab_total.index, 
                     y = ab_total.buyers,
                     name = "buyers",
                     text = ab_total.buyers,
                     hovertemplate="%{x}, <br>%{y} <extra></extra>",
                     textposition='auto',
                     marker=dict(color=['teal', 'salmon'])
                    ), row = 1, col = 2)
fig.add_trace(go.Bar(x = ab_total.index, 
                     y = ab_total.buyers_users_conv,
                     name = "conversion",
                     text = [str(round(i*100,2))+" %" for i in ab_total.buyers_users_conv],
                     hovertemplate="%{x}, <br>%{y:.2%} <extra></extra>",
                     textposition='auto',
                     marker=dict(color=['teal', 'salmon'])
                    ), row = 1, col = 3)
fig.add_trace(go.Bar(x = ab_total.index, 
                     y = ab_total.amount,
                     name = "revenue",
                     text = [str(round(i))+" $" for i in ab_total.amount],
                     hovertemplate="%{x}, <br>%{y} <extra></extra>",
                     textposition='auto',
                     marker=dict(color=['teal', 'salmon'])
                    ), row = 1, col = 4)
fig.add_trace(go.Bar(x = ab_total.index, 
                     y = ab_total.avg_bill,
                     name = "avg_bill",
                     text = [str(round(i,2))+" $" for i in ab_total.avg_bill],
                     hovertemplate="%{x}, <br>%{y} <extra></extra>",
                     textposition='auto',
                     marker=dict(color=['teal', 'salmon'])
                    ), row = 2, col = 1)
fig.add_trace(go.Bar(x = ab_total.index, 
                     y = ab_total.ARPU,
                     name = "ARPU",
                     text = [str(round(i, 3))+" $" for i in ab_total.ARPU],
                     hovertemplate="%{x}, <br>%{y} <extra></extra>",
                     textposition='auto',
                     marker=dict(color=['teal', 'salmon'])
                    ), row = 2, col = 2)
fig.add_trace(go.Bar(x = ab_total.index, 
                     y = ab_total.ARPPU,
                     name = "ARPPU",
                     text = [str(round(i, 2))+" $" for i in ab_total.ARPPU],
                     hovertemplate="%{x}, <br>%{y} <extra></extra>",
                     textposition='auto',
                     marker=dict(color=['teal', 'salmon'])
                    ), row = 2, col = 3)

fig.update_layout(height=500, width=980,
                  showlegend=False,
                  title_text="Результаты А/В теста на разные стартовые истории при первом запуске приложения")

fig.show()

<blockquote>
<p><b>Вывод: </b>Итак, мы получили отчет по А/В тесту с интересующими нас метриками. По нему можно ответить на вопросы, связанные с метриками нашего теста и сделать выводы относительно его успешности:<br><br> 1. Конверсия статистически незначимо уменьшилась на <b>17 %</b><br> 2. Средний чек статистически незначимо увеличился на <b>20 %</b><br> 3. ARPU статистически незначимо остался практически неизменным (увеличился на <b>0,5 %</b><br> 4. ARPPU статистически незначимо увеличился на <b>21 %</b> <br><br><b>Рекомендации:</b> запустить тест на большую аудиторию, чтобы получить статистически значимые результаты по всем рассматриваемым метрикам. По полученным результатам (если не принимать во внимание их несоответсвие на статистическую значимость), в виду лучших монетизационных показателей среднего чека, ARPU (пусть и очень незначительного), ARPPU рекомендуется оставить тестовый вариант стартовых историй при первом запуске как более интересный для нашей целевой аудитории. Уменьшение конверсии в тестовой группе наводит на мысль об анализе трафика с которого приходят наши пользователи - возможно не лучшим образом охватывает пользователей, которым интересно наше приложение.</p>
</blockquote>